In [1]:
#define a new DFT which is unitary
#NOTE: in Beals' ['97] he normalizes by \sqrt{d_\lambda/n!}
#but also notes that a basis change is an equivalence relation on rep'ns
#and each equivalence class contains a unitary representatione
#if each rep'n \rho \in \hat{G} is unitary, then the transformation is unitary
#these representations are not unitary
#to make them unitary, use Weyl's unitary trick
#OPTION 1: use the formula P = \int_G \rho(g)\rho(g)^* dg, and take a square root to find Q s.t. P = Q^2
#OPTION 2: define a new invariant inner product compute an orthonormal basis w.r.t. 
#this inner product by computing the Gram matrix A and using A.gram_schmidt()

In [138]:
#define conjugation as x |--> x**q, an order two automorphism of F_q^2. note x**q == x for x \in F_q.
def conjugate_transpose_pos_char(A):
    assert A.nrows() == A.ncols()
    field_size = A.base_ring().order()
    q = sqrt(field_size) if field_size.is_square() else field_size
    print(q)
    return matrix(GF(q**2),[[A[i][j]**q for i in range(A.nrows())] for j in range(A.nrows())]).transpose()

In [86]:
#find the change-of-basis matrix Q making \rho(g) unitary for all g \in G
def unitary_change_of_basis(SGA,partition):
    rho = SGA.specht_module(partition).representation_matrix
    group_size = SGA.group().cardinality()
    if SGA.characteristic() > 0:
        P = SGA.base_ring()(1/group_size)*sum(rho(g)*conjugate_transpose_pos_char(rho(g)) for g in SGA.group())
        D, M = P.diagonalization()
        sqrt_D = diagonal_matrix([P.base_ring()(D[i][i]).square_root() for i in range(D.ncols())])
        return M*sqrt_D*M.inverse()
    else:
        P = (1/group_size)*sum(rho(g)*rho(g).conjugate().transpose() for g in SGA.group())
        return P.principal_square_root()

In [87]:
#define the Fourier coefficient at the representation specht_module
#which is the Specht module corresponding to partition
def hat(f,partition,SGA,unitary=False):
    specht_module = SGA.specht_module(partition)
    rho = specht_module.representation_matrix
    if unitary:
        Q = unitary_change_of_basis(SGA,partition)
        F = Q.base_ring()
        unitary_factor = F(specht_module.dimension()/SGA.group().cardinality()).square_root()
        return unitary_factor*sum(f(g)*Q.inverse()*rho(g)*Q for g in SGA.group())
    else:
        return sum(f(g)*rho(g) for g in SGA.group())

In [88]:
#define the delta function delta_s(t) = {1 if s == t, 0 otherwise}
delta = lambda s: lambda t: 1 if t == s else 0

In [89]:
#for each basis element g \in G compute the Fourier coefficients \hat{\delta_g}(partition) for all partitions
from sage.misc.flatten import flatten
def unitary_dft(SGA):
    return matrix([flatten([hat(delta(g),partition,SGA,unitary=True).list() for partition in Partitions(SGA.group().degree())]) for g in G]).transpose()

In [55]:
#QUESTION: is the SGA DFT unitary?

In [62]:
SGA = SymmetricGroupAlgebra(GF(7),3)

In [63]:
G = SGA.group()

In [64]:
SGA_dft = SGA.dft(); SGA_dft

[1 1 1 1 1 1]
[1 4 6 3 3 4]
[0 6 0 6 1 1]
[0 1 0 6 1 6]
[1 3 1 3 3 3]
[1 6 6 1 1 6]

In [65]:
#check if A*A^T == Id. it's not, but the columns are orthonormal
SGA_dft*SGA_dft.transpose()

[6 0 0 0 0 0]
[0 3 0 0 0 0]
[0 0 4 0 0 0]
[0 0 0 4 0 0]
[0 0 0 0 3 0]
[0 0 0 0 0 6]

In [66]:
partition = Partitions(SGA.group().degree())[1]; partition

[2, 1]

In [67]:
specht_module = SGA.specht_module(partition)

In [68]:
rho = specht_module.representation_matrix

In [69]:
P = SGA.base_ring()(1/SGA.group().cardinality())*sum(rho(g)*conjugate_transpose_pos_char(rho(g)) for g in SGA.group()); P

[4 0]
[0 4]

In [70]:
q2 = P.base_ring().cardinality(); q2

49

In [71]:
D, M = P.diagonalization(); D

[4 0]
[0 4]

In [72]:
F = GF(q2)

In [73]:
sqrt_D = diagonal_matrix([F(D[i][i]).square_root() for i in range(D.ncols())]); sqrt_D

[2 0]
[0 2]

In [100]:
Q = M*sqrt_D*M.inverse(); Q^2 == P

True

In [99]:
Q = unitary_change_of_basis(SGA,partition); Q

[2 0]
[0 2]

In [137]:
G[3]

[2, 3, 1]

In [136]:
rho(G[3])

[6 1]
[6 0]

In [134]:
U_rho = Q.inverse()*rho(G[3])*Q; U_rho

[6 1]
[6 0]

In [139]:
U_rho*conjugate_transpose_pos_char(U_rho)

7


[0 6]
[1 6]

In [126]:
U_dft = unitary_dft(SGA); U_dft*conjugate_transpose_pos_char(U_dft)

[3 1 3 5 5 3]
[0 1 6 4 3 3]
[0 0 6 1 6 1]
[0 2 4 3 6 6]
[6 5 4 6 5 5]
[0 2 5 3 4 6]

In [96]:
#check that the DFT is unitary
(U_dft*U_dft.transpose()) == identity_matrix(SGA.group().cardinality())

False

In [92]:
#what are the eigenvalues?

In [94]:
eigs = matrix(CDF,unitary_dft(3).simplify_full()).eigenvalues()

AttributeError: 'sage.rings.integer.Integer' object has no attribute 'group'

In [ ]:
eigs

In [93]:
#n=3: two real, two complex
#n=4: all complex
#the magnitude is not 1, they're closely grouped around 2 or 3

In [ ]:
eigs = A.eigenvalues(); eigs

In [ ]:
[abs(eig) for eig in eigs]

In [ ]:
#note that the singluar values are the square roots of the diagonal entries of the Gram matrix
print(SymmetricGroup(n).algebra(CDF).dft().SVD()[1].numpy().diagonal())
print(sqrt((A*A.transpose()).numpy().diagonal()))